In [1]:
#######################################################################
#######################################################################
######################### Importing Packages ##########################
#######################################################################
#######################################################################

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import tensorflow as tf
import math
from datetime import datetime, timedelta
import scipy.stats

# need to clean these up later
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow_probability as tfp
import pandas as pd
from keras.models import Sequential
from tensorflow.keras import regularizers
from keras.layers import Activation, Input, Dense, LSTM, concatenate
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
tfd = tfp.distributions

from numpy.random import rand
from numpy import ix_
import os
from collections import defaultdict
np.random.seed(1337)

# Data Importing

In [2]:
data_path = "C:/Warwick Final Year/RAE/Processed Data/"


ARMA_GARCH = defaultdict(dict)

for which_series in ['DAX','NASDAQ','Nikkei']:
    for filename in os.listdir(data_path + which_series +'/ARMA_GARCH'):
            if filename.endswith('.csv'):
                ARMA_GARCH[which_series][filename[:-4 or None]] = pd.read_csv(data_path + 
                                                                     which_series +
                                                                     '/ARMA_GARCH/' + 
                                                                     filename)

In [3]:
#######################################################################
#######################################################################
###################### Data Importing/Cleaning ########################
#######################################################################
#######################################################################

# import data and calculate log returns from adjusted close
df_Nikkei_RAW = pd.read_csv("C:/Warwick Final Year/RAE/Data/^N225.csv")
df_NASDAQ_RAW = pd.read_csv("C:/Warwick Final Year/RAE/Data/^IXIC.csv")
df_DAX_RAW = pd.read_csv("C:/Warwick Final Year/RAE/Data/^GDAXI.csv")

In [4]:
def Data_Processor(DATA,
                   Batch_Size,
                   Length_of_Batch,
                   Test_Train_Split):
    
    DATA.columns = [c.replace(' ', '_') for c in DATA.columns]
    DATA = DATA[DATA['Adj_Close'].notnull()]
    DATA['log_ret'] = np.log(DATA.Adj_Close) - np.log(DATA.Adj_Close.shift(1))

    # spilt to training and test sets
    DATA    = DATA[['Date','log_ret']][1:]
    DATA['Date'] = DATA['Date'].apply(pd.Timestamp)
    DATA.set_index('Date',inplace=True, drop=True)
    
    train = DATA.loc[:Test_Train_Split]
    test  = DATA.loc[Test_Train_Split:]
   
    DATA_train = [[i] for i in train['log_ret']]
    DATA_test  = [[i] for i in test['log_ret']]
    
    scaler = MinMaxScaler()
    scaler.fit(train)
    scaled_train = scaler.transform(train)
    scaled_test = scaler.transform(test)
    
    # creating time series generator for processing
    # make one for training, one for validation
    time_series_generator = TimeseriesGenerator(DATA_train, 
                                                DATA_train, 
                                                length=Length_of_Batch, 
                                                batch_size=Batch_Size)
    time_series_val_generator = TimeseriesGenerator(DATA_test,
                                                    DATA_test, 
                                                    length=Length_of_Batch, 
                                                    batch_size=Batch_Size)

    
    return {"Data" : DATA,
            "Training" : time_series_generator,
            "Validation" : time_series_val_generator}



In [5]:
Batch_Size = 64
Length = 10
Test_Train_Split = '2015-01-01'



Nikkei = Data_Processor(df_Nikkei_RAW,
                       Batch_Size,
                       Length,
                       Test_Train_Split)
NASDAQ = Data_Processor(df_NASDAQ_RAW,
                       Batch_Size,
                       Length,
                       Test_Train_Split)
DAX = Data_Processor(df_DAX_RAW,
                       Batch_Size,
                       Length,
                       Test_Train_Split)


DATA = {'Nikkei' : Nikkei,
        'NASDAQ' : NASDAQ,
        'DAX'    : DAX}

<ipython-input-4-2335be0e7533>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA['log_ret'] = np.log(DATA.Adj_Close) - np.log(DATA.Adj_Close.shift(1))
<ipython-input-4-2335be0e7533>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA['log_ret'] = np.log(DATA.Adj_Close) - np.log(DATA.Adj_Close.shift(1))


In [6]:
ARMA_GARCH['DAX']['ARMA_11_GARCH_11'].head()

,Date,Norm_mu,Norm_std,T_mu,T_std,T_shape,Lap_mu,Lap_std
0,2015-01-02,0.000817,0.013669,0.000972,0.013833,8.271357,0.001154,0.014623
1,2015-01-05,0.001120,0.016151,0.001441,0.015999,8.271357,0.002016,0.016924
2,2015-01-06,0.001108,0.015346,0.001389,0.015337,8.271357,0.001792,0.016231
3,2015-01-07,0.001042,0.014639,0.001257,0.014737,8.271357,0.001458,0.015589
4,2015-01-08,0.000706,0.017035,0.000708,0.016789,8.271357,0.000377,0.017703


In [7]:
for which_series in ['DAX','NASDAQ','Nikkei']:
    for k in ARMA_GARCH[which_series].keys():
        
    
        means = ARMA_GARCH[which_series][k]['Norm_mu']
        scale   = ARMA_GARCH[which_series][k]['Norm_std']

        x = pd.date_range("2015-01-01", periods=len(means),freq="D")


        plt.plot(x,DATA[which_series]['Data'].loc['2015-01-01':]['log_ret'], color='#1f77b4',zorder=1)

        ppf = scipy.stats.norm.ppf

        SCORE = round(np.sum(np.log(scipy.stats.norm.pdf(DATA[which_series]['Data'].loc['2015-01-01':]['log_ret'],
                                                 loc = means,
                                                 scale = scale)))/len(DATA[which_series]['Data'].loc['2015-01-01':]['log_ret']),4)
        NAME = "Normal"

        s1 = plt.fill_between(x,np.add(means,ppf(0.95,loc=means,scale=scale)),
                              np.add(means,ppf(0.05,loc=means,scale=scale)), 
                              color='green',zorder=4,alpha=0.4)
        s2 = plt.fill_between(x,np.add(means,ppf(0.975,loc=means,scale=scale)),
                         np.add(means,ppf(0.025,loc=means,scale=scale)), 
                         color='grey',
                             zorder=3,
                             alpha=0.5)
        plt.plot(x,means, color='black',zorder=2)

        plt.title(NAME)

        plt.text(pd.to_datetime('2019-06-01'), 
                -0.08,
            "SCORE="+str(SCORE),
            horizontalalignment='center', 
                fontweight='bold', 
                color='red',
               fontsize='medium')

        years = mdates.YearLocator(10)   # every year
        years_fmt = mdates.DateFormatter('%Y')


        #plt.xaxis.set_major_locator(years)
        #plt.xaxis.set_major_formatter(years_fmt)
        plt.ylabel("Log Returns")
        plt.legend(handles=[s1,s2], labels=["10%","5%"],loc='upper right')
        plt.ylim([-0.15, 0.15])
        plt.tight_layout() 

        plt.savefig('C:/Warwick Final Year/RAE/Graphs/ARMA_GARCH_Graphs/' + 
                    which_series + "_" +
                    k +  "_" +
                    'Normal' + 
                    '.pdf')
        plt.close()


In [8]:
# next the Laplace
for which_series in ['DAX','NASDAQ','Nikkei']:
    for k in ARMA_GARCH[which_series].keys():
        
    
        means = ARMA_GARCH[which_series][k]['Lap_mu']
        scale   = ARMA_GARCH[which_series][k]['Lap_std']/2

        x = pd.date_range("2015-01-01", periods=len(means),freq="D")


        plt.plot(x,DATA[which_series]['Data'].loc['2015-01-01':]['log_ret'], color='#1f77b4',zorder=1)

        ppf = scipy.stats.laplace.ppf

        SCORE = round(np.sum(np.log(scipy.stats.laplace.pdf(DATA[which_series]['Data'].loc['2015-01-01':]['log_ret'],
                                                 loc = means,
                                                 scale = scale)))/len(DATA[which_series]['Data'].loc['2015-01-01':]['log_ret']),4)
        NAME = "Laplace"

        s1 = plt.fill_between(x,np.add(means,ppf(0.95,loc=means,scale=scale)),
                              np.add(means,ppf(0.05,loc=means,scale=scale)), 
                              color='green',zorder=4,alpha=0.4)
        s2 = plt.fill_between(x,np.add(means,ppf(0.975,loc=means,scale=scale)),
                         np.add(means,ppf(0.025,loc=means,scale=scale)), 
                         color='grey',
                             zorder=3,
                             alpha=0.5)
        plt.plot(x,means, color='black',zorder=2)

        plt.title(NAME)

        plt.text(pd.to_datetime('2019-06-01'), 
                -0.08,
            "SCORE="+str(SCORE),
            horizontalalignment='center', 
                fontweight='bold', 
                color='red',
               fontsize='medium')

        years = mdates.YearLocator(10)   # every year
        years_fmt = mdates.DateFormatter('%Y')


        #plt.xaxis.set_major_locator(years)
        #plt.xaxis.set_major_formatter(years_fmt)
        plt.ylabel("Log Returns")
        plt.legend(handles=[s1,s2], labels=["10%","5%"],loc='upper right')
        plt.ylim([-0.15, 0.15])
        plt.tight_layout() 

        plt.savefig('C:/Warwick Final Year/RAE/Graphs/ARMA_GARCH_Graphs/' + 
                    which_series + "_" +
                    k +  "_" +
                    'Laplace' + 
                    '.pdf')
        plt.close()

In [10]:
# finally the T
for which_series in ['DAX','NASDAQ','Nikkei']:
    for k in ARMA_GARCH[which_series].keys():
        
    
        shape = ARMA_GARCH[which_series][k]['T_shape'] 

        means = ARMA_GARCH[which_series][k]['T_mu']

        scale   = np.divide(ARMA_GARCH[which_series][k]['T_std'],np.sqrt(np.divide(shape,(shape-2))))

        x = pd.date_range("2015-01-01", periods=len(means),freq="D")


        plt.plot(x,DATA[which_series]['Data'].loc['2015-01-01':]['log_ret'], color='#1f77b4',zorder=1)

        ppf = scipy.stats.t.ppf

        SCORE = round(np.sum(np.log(scipy.stats.t.pdf(DATA[which_series]['Data'].loc['2015-01-01':]['log_ret'],
                                                 loc = means,
                                                 scale = scale,
                                                 df = shape)))/len(DATA[which_series]['Data'].loc['2015-01-01':]['log_ret']),4)
        NAME = "Student_T"

        s1 = plt.fill_between(x,np.add(means,ppf(0.95,loc=means,scale=scale,df=shape)),
                              np.add(means,ppf(0.05,loc=means,scale=scale,df=shape)), 
                              color='green',zorder=4,alpha=0.4)
        s2 = plt.fill_between(x,np.add(means,ppf(0.975,loc=means,scale=scale,df=shape)),
                         np.add(means,ppf(0.025,loc=means,scale=scale,df=shape)), 
                         color='grey',
                             zorder=3,
                             alpha=0.5)
        plt.plot(x,means, color='black',zorder=2)

        plt.title(NAME)

        plt.text(pd.to_datetime('2019-06-01'), 
                -0.08,
            "SCORE="+str(SCORE),
            horizontalalignment='center', 
                fontweight='bold', 
                color='red',
               fontsize='medium')

        years = mdates.YearLocator(10)   # every year
        years_fmt = mdates.DateFormatter('%Y')


        #plt.xaxis.set_major_locator(years)
        #plt.xaxis.set_major_formatter(years_fmt)
        plt.ylabel("Log Returns")
        plt.legend(handles=[s1,s2], labels=["10%","5%"],loc='upper right')
        plt.ylim([-0.15, 0.15])
        plt.tight_layout() 

        plt.savefig('C:/Warwick Final Year/RAE/Graphs/ARMA_GARCH_Graphs/' + 
                    which_series + "_" +
                    k +  "_" +
                    'T' + 
                    '.pdf')
        plt.close()